In [1]:
# launch XVFB if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

### Let's make a TRPO!

In this notebook we will write the code of the one Trust Region Policy Optimization.
As usually, it contains a few different parts which we are going to reproduce.



In [2]:
import numpy as np
import math
import theano
import theano.tensor as T
from math import radians

In [3]:
import gym
import gym_crumb
env = gym.make("crumb-synthetic-v0")


In [4]:
a = env.reset()
len(a)

13

### Step 1: Defining a network

With all it's complexity, at it's core TRPO is yet another policy gradient method. 

This essentially means we're actually training a stochastic policy $ \pi_\theta(a|s) $. 

And yes, it's gonna be a neural network. So let's start by defining one.

In [5]:
env.step([0,radians(180)])

((3.6739403974420594e-16,
  3.6739403974420594e-16,
  0.0,
  3.6739403974420594e-16,
  0.0,
  3.6739403974420594e-16,
  0.0,
  2.220446049250313e-16,
  2.220446049250313e-16,
  0.0,
  3.141592653589793,
  0.0,
  0.0),
 1059.0,
 True)

In [6]:
env.reset()

(6.0, 0.0, 6.0, 0.0, 6.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [7]:
#input tensors
observations = T.matrix(name="obs")
actions = T.matrix(name="action")
cummulative_returns = T.vector(name="G = r + gamma*r' + gamma^2*r'' + ...")
old_m = T.matrix(name="action probabilities from previous iteration m") 
old_logstd = T.matrix(name="action probabilities from previous iteration sigma") 
all_inputs = [observations, actions, cummulative_returns, old_m, old_logstd]

In [8]:
from lasagne.layers import *
from lasagne.nonlinearities import softmax, linear, elu, tanh, sigmoid
from lasagne.init import GlorotNormal, Constant, Normal

In [9]:
observation_shape = (len(a),)
n_actions = 3

In [10]:
# Create neural network.

nn = InputLayer((None,)+observation_shape, input_var=observations)
#dropout = DropoutLayer(nn, p=0.1)

nn1 = DenseLayer(nn,128, W = GlorotNormal())
#dropout1 = DropoutLayer(nn1, p=0.1)
nn2 = DenseLayer(nn1,64, W = GlorotNormal())
#dropout2 = DropoutLayer(nn2, p=0.1)
#policy = DenseLayer(nn1, n_actions, nonlinearity=softmax)

m = DenseLayer(nn2, n_actions, nonlinearity=linear)
logsigma = DenseLayer(nn2, n_actions, nonlinearity=linear)
mean = get_output([m])[0]
logstd = get_output([logsigma])[0]
#prob = T.prod(T.sqrt(1/(2*math.pi*sigma_m[1]))*T.exp(-(actions-sigma_m[0])**2/(2*sigma_m[1])),axis = -1)
#old_prob = T.prod(T.sqrt(1/(2*math.pi*old_sigma))*T.exp(-(actions-old_m)**2/(2*old_sigma)), axis = -1)
log_p = - T.sum(logstd, -1) - 0.5 * T.sum(T.square((actions - mean) / T.exp(logstd)), -1) - 0.5 * np.log(2 * np.pi)
oldlog_p = - T.sum(old_logstd, -1) - 0.5 * T.sum(T.square((actions - old_m) / T.exp(old_logstd)), -1) - 0.5 * np.log(2 * np.pi)
weights = get_all_params([m,logsigma],trainable=True)


### Step 2: Actions and rollouts

In this section, we'll define functions that take actions $ a \sim \pi_\theta(a|s) $ and rollouts $ <s_0,a_0,s_1,a_1,s_2,a_2,...s_n,a_n> $.

In [11]:
#compile function
get_mean = theano.function([observations], mean, allow_input_downcast=True)
get_logstd = theano.function([observations], logstd, allow_input_downcast=True)

from scipy.stats import multivariate_normal
def act(obs, sample=True):
    """
    Samples action from policy distribution (sample = True) or takes most likely action (sample = False)
    :param: obs - single observation vector
    :param sample: if True, samples from \pi, otherwise takes most likely action
    :returns: action (single integer) and probabilities for all actions
    """

    
    m = get_mean([obs])[0]
    
    log_std = get_logstd([obs])[0]
    rnd = np.random.normal(size = n_actions)
    if sample:
        action = np.exp(log_std)*rnd + m
    else:
        action = m

    return action, m, log_std


In [12]:
#demo
print ("sampled:", act(env.reset()))
print ("greedy:", act(env.reset(), sample=False))

sampled: (array([-1.21670069, -1.86442173,  0.73801826]), array([-0.95880413, -1.45205058,  0.43762131]), array([-1.50083619, -0.18878248, -1.33129248]))
greedy: (array([-0.95880413, -1.45205058,  0.43762131]), array([-0.95880413, -1.45205058,  0.43762131]), array([-1.50083619, -0.18878248, -1.33129248]))


Compute cummulative reward just like you did in vanilla REINFORCE

In [13]:
import scipy.signal
def get_cummulative_returns(r, gamma=0.9):
    """
    Computes cummulative discounted rewards given immediate rewards
    G_i = r_i + gamma*r_{i+1} + gamma^2*r_{i+2} + ...
    Also known as R(s,a).
    """
    r = np.array(r)
    assert r.ndim >= 1
    return scipy.signal.lfilter([1], [1, -gamma], r[::-1], axis=0)[::-1]

In [14]:
#simple demo on rewards [0,0,1,0,0,1]
get_cummulative_returns([0,0,1,0,0,1], gamma=0.9)

array([1.40049, 1.5561 , 1.729  , 0.81   , 0.9    , 1.     ])

**Rollout**

In [15]:
from math import radians
def rollout(env, act, max_pathlength=2500, n_timesteps=50000):
    """
    Generate rollouts for training.
    :param: env - environment in which we will make actions to generate rollouts.
    :param: act - the function that can return policy and action given observation.
    :param: max_pathlength - maximum size of one path that we generate.
    :param: n_timesteps - total sum of sizes of all pathes we generate.
    """
    paths = []

    total_timesteps = 0
    while total_timesteps < n_timesteps:
        obervations, actions, rewards, action_m, action_logstd = [], [], [], [], []
        obervation = env.reset()
        for _ in range(max_pathlength):
            action, m, logstd = act(obervation)
            obervations.append(obervation)
            actions.append(action)
            action_m.append(m)
            action_logstd.append(logstd)
            reward = 0
            for i in range(n_actions):
                obervation, r, done = env.step([i,action[i]])
                reward += r
            #obervation, reward, done = env.step([0,radians(180*action)])
            rewards.append(reward)
            total_timesteps += 1
            if done or total_timesteps==n_timesteps:
                path = {"observations": np.array(obervations),
                        "m": np.array(action_m),
                        "logstd": np.array(action_logstd),
                        "actions": np.array(actions),
                        "rewards": np.array(rewards),
                        "cumulative_returns":get_cummulative_returns(rewards),
                       }
                paths.append(path)
                break
    return paths

In [16]:
paths = rollout(env,act,max_pathlength=5,n_timesteps=5)
print (paths[-1])
print(paths[0]['m'].shape)
assert (paths[0]['cumulative_returns'].shape==(5,))
assert (paths[0]['rewards'].shape==(5,))
assert (paths[0]['observations'].shape==(5,)+observation_shape)
assert (paths[0]['actions'].shape==(5,n_actions))
print ('It\'s ok')

{'observations': array([[ 6.        ,  0.        ,  6.        ,  0.        ,  6.        ,
         0.        ,  6.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 1.62287434, -0.54797791,  1.52756059, -0.54797791,  1.52756059,
        -0.54797791,  1.52756059, -0.35627542, -0.07702618,  0.39144128,
         5.19001172,  3.86479911,  0.39144128],
       [ 4.52715362, -0.70140961,  4.4724875 , -0.70140961,  4.4724875 ,
        -0.70140961,  4.4724875 , -0.44454381,  0.43786208,  5.7437375 ,
         4.25074143,  2.40293028,  5.7437375 ],
       [ 4.72345237, -1.57863982,  4.45184215, -1.57863982,  4.45184215,
        -1.57863982,  4.45184215, -0.82721453, -0.00601767,  0.21465393,
         4.27896613,  1.55263763,  0.21465393],
       [ 0.17566345,  0.15172587,  0.08852631,  0.15172587,  0.08852631,
         0.15172587,  0.08852631,  0.05206598,  0.07968042,  5.7153632 ,
         3.16657259,  0.16676462,  5.7153632 ]]), 'm': array(

### Step 3: loss functions

Now let's define the loss functions and constraints for actual TRPO training.

The surrogate reward should be
$$J_{surr}= {1 \over N} \sum\limits_{i=0}^N \frac{\pi_{\theta}(s_i, a_i)}{\pi_{\theta_{old}}(s_i, a_i)}A_{\theta_{old}(s_i, a_i)}$$

For simplicity, let's use cummulative returns instead of advantage for now:
$$J'_{surr}= {1 \over N} \sum\limits_{i=0}^N \frac{\pi_{\theta}(s_i, a_i)}{\pi_{\theta_{old}}(s_i, a_i)}G_{\theta_{old}(s_i, a_i)}$$

Or alternatively, minimize the surrogate loss:
$$ L_{surr} = - J'_{surr} $$

In [17]:
#select probabilities of chosen actions
batch_size=actions.shape[0]

#m_for_actions_m = prob[T.arange(batch_size), actions]
#old_probs_for_actions = old_probs[T.arange(batch_size), actions]


In [18]:
# Compute surrogate loss: negative importance-sampled policy gradient

L_surr = -T.sum(T.exp(log_p - oldlog_p)*cummulative_returns)#<compute surrogate loss, aka _negative_ importance-sampled policy gradient>

In [19]:
#compute and return surrogate policy gradient

def get_flat_gradient(loss, var_list):
    """gradient of loss wrt var_list flattened into a large vector"""
    grads = T.grad(loss, var_list)
    return T.concatenate([grad.ravel() for grad in grads])

get_surrogate_gradients = theano.function(all_inputs, get_flat_gradient(L_surr, weights),
                                          allow_input_downcast=True)

We can ascend these gradients as long as our $pi_\theta(a|s)$ satisfies the constraint
$$E_{s,\pi_{\Theta_{t}}}\Big[KL(\pi(\Theta_{t}, s) \:||\:\pi(\Theta_{t+1}, s))\Big]< \alpha$$


where

$$KL(p||q) = E _p log({p \over q})$$

In [20]:
# Compute Kullback-Leibler divergence (see formula above)
# Note: you need to sum KL and entropy over all actions, not just the ones agent took
numerator = (old_m - mean)**2 + T.exp(2*old_logstd) - T.exp(2*logstd)
denominator = 2 * T.exp(2*logstd) + 1e-8
kl = T.mean(T.sum((numerator/denominator + 0.5*logstd - 0.5*old_logstd), axis=-1))#<cumpute kullback-leibler as per formula above>
#Compute policy entropy
entropy = -T.sum(logstd + 0.5*np.log(2*math.pi*math.e))

compute_losses = theano.function(all_inputs,[L_surr, kl, entropy],
                                 allow_input_downcast=True)

**Linear search**

TRPO in its core involves ascending surrogate policy gradient constrained by KL divergence. 

In order to enforce this constraint, we're gonna use linesearch. You can find out more about it [here](https://en.wikipedia.org/wiki/Linear_search)

In [21]:
def linesearch(f, x, fullstep, max_kl):
    """
    Linesearch finds the best parameters of neural networks in the direction of fullstep contrainted by KL divergence.
    :param: f - function that returns loss, kl and arbitrary third component.
    :param: x - old parameters of neural network.
    :param: fullstep - direction in which we make search.
    :param: max_kl - constraint of KL divergence.
    :returns:
    """
    max_backtracks = 10
    loss, _, _ = f(x)
    for stepfrac in .5 ** np.arange(max_backtracks):
        xnew = x + stepfrac * fullstep
        new_loss, kl, _ = f(xnew)
        actual_improve = new_loss - loss
        if kl<=max_kl and actual_improve < 0:
            x = xnew
            loss = new_loss
    return x

### Step 4: training
In this section we construct rest parts of our computational graph

In [22]:
def slice_vector(vector,shapes):
    """
    Slices symbolic vector into several symbolic tensors of given shapes.
    Auxilary function used to un-flatten gradients, tangents etc.
    :param vector: 1-dimensional symbolic vector
    :param shapes: list or tuple of shapes (list, tuple or symbolic)
    :returns: list of symbolic tensors of given shapes
    """
    assert vector.ndim==1,"vector must be 1-dimensional"
    start = 0
    tensors = []
    for shape in shapes:
        size = T.prod(shape)
        tensor = vector[start:(start + size)].reshape(shape)
        tensors.append(tensor)
        start += size
    return tensors

In [23]:
conjugate_grad_intermediate_vector = T.vector("intermediate grad in conjugate_gradient")

#slice flat_tangent into chunks for each weight
weight_shapes = [var.get_value().shape for var in weights]
tangents = slice_vector(conjugate_grad_intermediate_vector,weight_shapes)

# KL divergence where first arg is fixed
from theano.gradient import disconnected_grad as const
numerator_firstfixed = (const(mean) - mean)**2 + const(T.exp(logstd)) - T.exp(logstd)
denominator_firstfixed = 2 * T.exp(logstd) + 1e-8
kl_firstfixed = T.mean(T.sum((numerator_firstfixed/denominator_firstfixed + 0.5*logstd - 0.5*const(logstd)), axis = -1))
#kl_firstfixed = (const(probs) * (const(T.log(probs)) - T.log(probs))).sum(axis=-1).mean()

#compute fisher information matrix (used for conjugate gradients and to estimate KL)
gradients = T.grad(kl_firstfixed, weights)
gradient_vector_product = [T.sum(g * t) for (g, t) in zip(gradients, tangents)]

fisher_vector_product = get_flat_gradient(sum(gradient_vector_product), weights)

compute_fisher_vector_product = theano.function([observations, conjugate_grad_intermediate_vector],fisher_vector_product, allow_input_downcast=True)

### TRPO helpers

Here we define a few helper functions used in the main TRPO loop

**Conjugate gradients**

Since TRPO includes contrainted optimization, we will need to solve Ax=b using conjugate gradients.

In general, CG is an algorithm that solves Ax=b where A is positive-defined. A is Hessian matrix so A is positive-defined. You can find out more about them [here](https://en.wikipedia.org/wiki/Conjugate_gradient_method)

In [24]:
from numpy.linalg import inv
def conjugate_gradient(f_Ax, b, cg_iters=10, residual_tol=1e-10):
    """
    This method solves system of equation Ax=b using iterative method called conjugate gradients
    :f_Ax: function that returns Ax
    :b: targets for Ax
    :cg_iters: how many iterations this method should do
    :residual_tol: epsilon for stability
    """
    p = b.copy()
    r = b.copy()
    x = np.zeros_like(b)
    rdotr = r.dot(r)
    for i in range(cg_iters):
        z = f_Ax(p)
        v = rdotr / (p.dot(z) + 1e-8)
        x += v * p
        r -= v * z
        newrdotr = r.dot(r)
        mu = newrdotr / (rdotr + 1e-8)
        p = r + mu * p
        rdotr = newrdotr
        if rdotr < residual_tol:
            break
    return x

In [25]:
#This code validates conjugate gradients
A = np.random.rand(8, 8)
A = np.matmul(np.transpose(A), A)

def f_Ax(x):
    return np.matmul(A, x.reshape(-1, 1)).reshape(-1)

b = np.random.rand(8)

w = np.matmul(np.matmul(inv(np.matmul(np.transpose(A), A)), np.transpose(A)), b.reshape((-1, 1))).reshape(-1)
print (w)
print (conjugate_gradient(f_Ax, b))

[ -89.85281809   30.39316113   89.41903467  -24.71419458 -150.29023183
  133.0692808   -20.78374787   73.81775525]
[ -89.85179926   30.39395495   89.41948507  -24.71507106 -150.28966893
  133.06836329  -20.78399333   73.81658756]


In [26]:
#Compile a function that exports network weights as a vector
flat_weights = T.concatenate([var.ravel() for var in weights])
get_flat_weights = theano.function([],flat_weights) 

#... and another function that imports vector back into network weights
flat_weights_placeholder = T.vector("flattened weights")
assigns = slice_vector(flat_weights_placeholder, weight_shapes)

load_flat_weights = theano.function([flat_weights_placeholder],updates=dict(zip(weights, assigns)))

/home/airan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The parameter 'updates' of theano.function() expects an OrderedDict, got <class 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (theano.compat.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
  if __name__ == '__main__':


##### Step 5: Main TRPO loop

Here we will train our network!

In [27]:
import time
from itertools import count
from collections import OrderedDict

max_kl=0.01           #this is hyperparameter of TRPO. It controls how big KL divergence may be between old and new policy every step.
cg_damping=0.1        #This parameters regularize addition to
numeptotal = 0        #this is number of episodes that we played.

start_time = time.time()

for i in count(1):
    print ("\n********** Iteration %i ************" % i)

    # Generating paths.
    print("Rollout")
    paths = rollout(env,act,max_pathlength=5000,n_timesteps=5000)
    print ("Made rollout")
    
    # Updating policy.
    observations = np.concatenate([path["observations"] for path in paths])
    actions = np.concatenate([path["actions"] for path in paths])
    returns = np.concatenate([path["cumulative_returns"] for path in paths])
    old_m = np.concatenate([path["m"] for path in paths])
    old_logstd = np.concatenate([path["logstd"] for path in paths])
    inputs_batch=[observations,actions,returns,old_m, old_logstd]
    
    old_weights = get_flat_weights()
    
    def fisher_vector_product(p):
        """gets intermediate grads (p) and computes fisher*vector """
        return compute_fisher_vector_product(observations, p) + cg_damping * p

    flat_grad = get_surrogate_gradients(*inputs_batch)
    
    stepdir = conjugate_gradient(fisher_vector_product, -flat_grad)
    shs = .5 * stepdir.dot(fisher_vector_product(stepdir))
    lm = np.sqrt(shs / max_kl)
    fullstep = stepdir / lm
    #Compute new weights with linesearch in the direction we found with CG
    
    def losses_f(flat_weights):
        load_flat_weights(flat_weights)
        return compute_losses(*inputs_batch)

    new_weights = linesearch(losses_f, old_weights, fullstep, max_kl)
    
    load_flat_weights(new_weights)

    #Report current progress
    L_surr, kl, entropy = compute_losses(*inputs_batch)
    episode_rewards = np.array([path["rewards"].sum() for path in paths])

    stats = OrderedDict()
    numeptotal += len(episode_rewards)
    stats["Total number of episodes"] = numeptotal
    stats["Average sum of rewards per episode"] = episode_rewards.mean()
    stats["Std of rewards per episode"] = episode_rewards.std()
    stats["Entropy"] = entropy
    stats["Time elapsed"] = "%.2f mins" % ((time.time() - start_time)/60.)
    stats["KL between old and new distribution"] = kl
    stats["Surrogate loss"] = L_surr
    if episode_rewards.mean() > 1000:
        break
    if np.isnan(L_surr):
        print (inputs_batch)
        break
    for k, v in stats.items():
        print(k + ": " + " " * (40 - len(k)) + str(v))
    i += 1




********** Iteration 1 ************
Rollout
Made rollout
Total number of episodes:                 2
Average sum of rewards per episode:       -23605.5
Std of rewards per episode:               3877.5
Entropy:                                  -14187.568500162231
Time elapsed:                             0.07 mins
KL between old and new distribution:      0.009969187906675612
Surrogate loss:                           464373.79386587744

********** Iteration 2 ************
Rollout
Made rollout
Total number of episodes:                 5
Average sum of rewards per episode:       -14026.333333333334
Std of rewards per episode:               10188.385293504016
Entropy:                                  -14278.533570170892
Time elapsed:                             0.14 mins
KL between old and new distribution:      0.009965219359255177
Surrogate loss:                           405453.27450418606

********** Iteration 3 ************
Rollout
Made rollout
Total number of episodes:              

Made rollout
Total number of episodes:                 88
Average sum of rewards per episode:       -3276.125
Std of rewards per episode:               3529.6795335235465
Entropy:                                  -13625.415889366126
Time elapsed:                             1.39 mins
KL between old and new distribution:      0.00991816889437115
Surrogate loss:                           249446.2607641903

********** Iteration 20 ************
Rollout
Made rollout
Total number of episodes:                 94
Average sum of rewards per episode:       -4027.8333333333335
Std of rewards per episode:               3462.9017897646986
Entropy:                                  -13597.988141799213
Time elapsed:                             1.46 mins
KL between old and new distribution:      0.009936013559514095
Surrogate loss:                           226344.11540421002

********** Iteration 21 ************
Rollout
Made rollout
Total number of episodes:                 102
Average sum of rewards 

In [29]:
def random_aim():
    x = np.random.choice([1,0,-1])
    if x != 0:
        y = np.random.choice([1,0,-1])
    else:
        y = np.random.choice([1,-1])
    aim = np.array([x,y])
    return aim/aim.sum()

In [30]:
random_aim()

array([-0.,  1.])

# Homework option I: better sampling (10+pts)

In this section, you're invited to implement a better rollout strategy called _vine_.

![img](https://s17.postimg.org/i90chxgvj/vine.png)

In most gym environments, you can actually backtrack by using states. You can find a wrapper that saves/loads states in [the mcts seminar](https://github.com/yandexdataschool/Practical_RL/blob/master/yet_another_week/seminar_MCTS.ipynb).

You can read more about in the [TRPO article](https://arxiv.org/abs/1502.05477) in section 5.2.

The goal here is to implement such rollout policy (we recommend using tree data structure like in the seminar above).
Then you can assign cummulative rewards similar to `get_cummulative_rewards`, but for a tree.

__bonus task__ - parallelize samples using multiple cores

# Homework option II (10+pts)

Let's use TRPO to train evil robots! (pick any of two)
* [MuJoCo robots](https://gym.openai.com/envs#mujoco)
* [Box2d robot](https://gym.openai.com/envs/BipedalWalker-v2)

The catch here is that those environments have continuous action spaces. 

Luckily, TRPO is a policy gradient method, so it's gonna work for any parametric $\pi_\theta(a|s)$. We recommend starting with gaussian policy:

$$\pi_\theta(a|s) = N(\mu_\theta(s),\sigma^2_\theta(s)) = {1 \over \sqrt { 2 \pi {\sigma^2}_\theta(s) } } e^{ (a - 
\mu_\theta(s))^2 \over 2 {\sigma^2}_\theta(s) } $$

In the $\sqrt { 2 \pi {\sigma^2}_\theta(s) }$ clause, $\pi$ means ~3.1415926, not agent's policy.

This essentially means that you will need two output layers:
* $\mu_\theta(s)$, a dense layer with linear activation
* ${\sigma^2}_\theta(s)$, a dense layer with activation T.exp (to make it positive; like rho from bandits)

For multidimensional actions, you can use fully factorized gaussian (basically a vector of gaussians).

__bonus task__: compare performance of continuous action space method to action space discretization

In [31]:
obs, _, _ = env.step([0, radians(a)])

TypeError: must be real number, not tuple

In [66]:
env.aim

array([-3,  0])

In [103]:
obs = env.reset()
env.aim = np.array([0, -2])
done = False
reward = 0
l = 0
while(done != True):
    a = act(obs,sample = True)[0]
    for i in range(n_actions):
                obs, r, done = env.step([i,a[i]])
    reward += r
    l += 1
    print (l,')',r, env.state) 
    
   

1 ) 13.0 [5.33512501 0.41032218 2.2454101 ]
2 ) 19.0 [4.55161201 5.18931122 4.82702502]
3 ) -2.0 [4.03674165 5.3704741  5.67472978]
4 ) -10.0 [3.81185219 5.65155344 3.58197761]
5 ) -4.0 [3.60032574 5.76575332 3.05298264]
6 ) 2.0 [3.35083779 0.52748205 3.24089662]
7 ) -8.0 [3.23999802 0.90503138 2.76151404]
8 ) -5 [2.94346364 0.44121177 2.72087431]
9 ) 7.0 [3.03906841 1.18393713 4.92260904]
10 ) -2.0 [2.74279999 1.29895806 5.11481714]
11 ) -8.0 [2.65768404 1.0462278  3.3420181 ]
12 ) -30.0 [3.5129132  0.87015618 1.16725885]
13 ) -5 [3.59766686 0.69612531 1.34236767]
14 ) -5 [3.30109874 0.21202857 1.52687339]
15 ) -22.0 [2.66931898 5.91579411 0.35646466]
16 ) -5 [3.13671622 6.2772611  2.34995703]
17 ) -5 [2.71530157 5.66089325 2.16179777]
18 ) -2.0 [3.16137929 6.17160914 5.71858032]
19 ) -8.0 [3.2415498  0.32973674 0.29304039]
20 ) 2.0 [2.58311794 1.18515429 5.77742401]
21 ) -4.0 [2.65771762 1.06799326 0.23659214]
22 ) -14.0 [2.52243365 1.06742987 3.10330539]
23 ) 11.0 [2.09990491 1.2133

206 ) -6.0 [2.73396663 0.58259718 3.55634952]
207 ) -32.0 [3.38617353 0.80162141 0.81928099]
208 ) 7.0 [3.33468218 0.91357082 2.59769927]
209 ) -20.0 [2.0199903  0.0606249  0.60105596]
210 ) 1.0 [2.38258553 0.73340222 0.76445416]
211 ) -6.0 [2.70454832 0.7314317  4.2655174 ]
212 ) -26.0 [2.77141344 1.41044699 0.69136991]
213 ) -2.0 [2.63646955 1.75940318 1.02507174]
214 ) 17.0 [3.04341113 1.73179078 3.51687792]
215 ) -26.0 [3.08434878 1.54392001 1.95024203]
216 ) 7.0 [2.87492228 1.3972703  5.45836836]
217 ) -14.0 [2.50628896 1.22812635 2.7564595 ]
218 ) -5 [3.77380087 0.72235086 2.74654495]
219 ) -2.0 [3.01662853 6.03147349 3.02001469]
220 ) -14.0 [3.32032322 0.67941414 1.90621489]
221 ) -2.0 [3.09840174 0.98691182 1.51744516]
222 ) -5 [3.06365063 0.83248875 1.73290304]
223 ) 2.0 [2.927289   0.66803327 0.12774006]
224 ) -4.0 [2.74900299 0.40525061 3.05332558]
225 ) 4.0 [2.93599155 0.79430634 4.20692305]
226 ) -22.0 [3.03303217 0.80249928 1.11245581]
227 ) -24.0 [2.46949699 0.60465652 3

414 ) -4.0 [2.62484805 1.25237754 5.93373147]
415 ) -6.0 [2.4698502  1.02084125 5.3194146 ]
416 ) -14.0 [2.79432507 1.04812936 1.05355805]
417 ) -5 [2.90709629 0.49757167 2.59553484]
418 ) -22.0 [3.90619251 0.68327262 0.70033702]
419 ) 11.0 [3.87804606 0.22026324 2.9822614 ]
420 ) -12.0 [3.46575418 0.35362998 1.98876763]
421 ) 3.0 [3.80538164 5.90161705 2.91365039]
422 ) -14.0 [2.76718707 0.05551179 4.54622877]
423 ) -5 [3.32702752 5.9283985  0.32743437]
424 ) -2.0 [3.26420982 0.03053922 1.0470211 ]
425 ) 4.0 [2.5066165  6.25850458 1.47159252]
426 ) -6.0 [2.70179398 0.528855   2.4455403 ]
427 ) -26.0 [2.33622564 5.93296642 0.45557657]
428 ) -12.0 [2.74927289 6.25483677 5.16984628]
429 ) -14.0 [3.15630199 0.58154355 2.20463775]
430 ) 1.0 [3.00441815 0.47109236 2.92229497]
431 ) -2.0 [3.09763958 6.27560052 5.77098891]
432 ) -10.0 [3.3864595  0.21371028 2.29638732]
433 ) 6.0 [3.42952331 6.07120483 3.96320659]
434 ) 1.0 [3.0631713  6.20460609 6.19586422]
435 ) 2.0 [3.3110697  6.19454541 4.

637 ) -5 [2.63073296 0.87608469 0.22453017]
638 ) -28.0 [2.08555718 1.54687965 3.92875458]
639 ) -5 [2.38459369 1.69047807 3.8497015 ]
640 ) 2.0 [2.51496072 1.49678373 5.95033905]
641 ) -14.0 [2.4451431  1.22786036 2.60273404]
642 ) -18.0 [3.44695421 1.18757757 1.04111175]
643 ) 7.0 [3.2641733  1.61130879 2.32716661]
644 ) -4.0 [3.39092031 2.38751679 4.39241866]
645 ) 3.0 [2.89988154 2.06245975 3.98551083]
646 ) -28.0 [2.65762264 1.84803108 2.03950785]
647 ) 7.0 [2.74829475 1.02077017 4.66519689]
648 ) -32.0 [2.99597041 1.38939387 0.84902037]
649 ) -5 [3.03625857 1.07645535 1.41552582]
650 ) 4.0 [2.7303618  1.15291434 3.43039547]
651 ) 1.0 [2.6261052  0.94550055 2.43903457]
652 ) -28.0 [2.08156764 0.62045418 4.85150167]
653 ) -16.0 [2.98538227 0.79387114 2.191851  ]
654 ) -32.0 [2.48122626 6.13265617 0.09108034]
655 ) 3.0 [3.21379826 0.68207789 2.75594875]
656 ) -4.0 [3.099197   6.05079167 3.41798575]
657 ) 5.0 [2.85580246 5.77396017 1.16808075]
658 ) -16.0 [2.98661729 6.15480274 4.382

876 ) -4.0 [3.50828767 0.06756125 1.60217109]
877 ) 2.0 [3.46562875 5.94903993 0.66486303]
878 ) 3.0 [3.18249248 0.16369404 5.26518875]
879 ) 2.0 [3.22828655 0.31630289 4.64592176]
880 ) -20.0 [3.37047717 0.21755786 0.0296696 ]
881 ) -5 [3.05170213 1.12544802 2.22171244]
882 ) 8.0 [2.80730536 1.17497875 4.03129918]
883 ) -5 [2.97104304 0.69124885 4.0525918 ]
884 ) -26.0 [3.16877193 0.67581415 1.72866055]
885 ) 3.0 [2.43399054 0.97188039 0.43868468]
886 ) -16.0 [2.37993374 1.27416257 2.67794163]
887 ) 8.0 [2.35252183 1.78180988 4.08661644]
888 ) -5 [2.68153031 2.0037796  4.2847676 ]
889 ) -22.0 [2.26651824 2.00398573 2.64853744]
890 ) -30.0 [3.20392718 2.52293086 5.87975075]
891 ) -2.0 [3.47730205 2.1190891  6.22064641]
892 ) 14.0 [2.94861709 1.69409013 3.67798458]
893 ) 6.0 [2.61891344 1.87101249 4.30901327]
894 ) -40.0 [2.49611127 2.2433234  0.90038737]
895 ) 4.0 [2.28533764 3.40082224 1.89624222]
896 ) -22.0 [3.50485649 3.89904053 0.68014193]
897 ) 9.0 [4.55175886 5.79713863 5.738058

1144 ) -16.0 [2.14492118 0.19093705 0.59554861]
1145 ) -2.0 [3.16371519 0.20937934 0.87860711]
1146 ) -5 [2.83920866 0.2865585  0.78191937]
1147 ) -6.0 [3.02801874 0.07328792 5.46083816]
1148 ) -2.0 [3.11029908 6.24170722 4.52677533]
1149 ) -12.0 [3.11300639 0.35981604 0.80679808]
1150 ) -18.0 [2.56604938 0.56410835 5.29316609]
1151 ) 1.0 [3.04763082 0.44960679 4.15956475]
1152 ) -6.0 [3.11597739 0.60608435 5.18608184]
1153 ) -5 [2.98185505 0.29618581 5.38184006]
1154 ) 3.0 [2.97208045 0.04424026 2.93913921]
1155 ) 4.0 [2.95955249 5.83474571 1.45596713]
1156 ) -22.0 [2.92186908 6.13377034 4.2446558 ]
1157 ) -5 [3.0005222  0.24107548 5.31906345]
1158 ) -6.0 [3.08904233 0.26416073 2.37920894]
1159 ) 1.0 [3.13857486 6.04164754 2.08283606]
1160 ) -18.0 [2.85365428 0.04425219 5.652542  ]
1161 ) -5 [3.28316005 6.00956092 5.51135877]
1162 ) -8.0 [3.50704119 0.14820953 6.04729225]
1163 ) -5 [3.06079424 0.20544923 5.54620428]
1164 ) 2.0 [3.1210834  6.21452364 1.48935393]
1165 ) -6.0 [3.0901419 

1371 ) 2.0 [2.01951751 0.70847513 2.19019271]
1372 ) 10.0 [2.11550908 1.18456569 0.92751573]
1373 ) -20.0 [1.71637193 0.47491014 6.09777797]
1374 ) -4.0 [2.612386   0.34012091 5.68251608]
1375 ) 1.0 [3.03635537 0.35858732 4.4830156 ]
1376 ) -4.0 [3.22978331 0.1668599  5.44267883]
1377 ) -5 [3.11409061 0.22386149 4.38771537]
1378 ) -5 [3.36597247 5.86318666 6.01473379]
1379 ) 7.0 [3.50036434 6.19820355 4.48798566]
1380 ) -2.0 [3.51509601 0.23441329 4.60374845]
1381 ) -5 [3.19508536 6.03048838 4.94930007]
1382 ) -18.0 [3.44170881 6.12989198 1.86934256]
1383 ) -5 [3.10243744 6.2520645  1.44941593]
1384 ) -8.0 [2.66636457 0.21485127 2.49037721]
1385 ) -36.0 [2.02676125 0.08338907 5.14716724]
1386 ) 3.0 [2.7591263  0.41552444 0.01820167]
1387 ) 10.0 [1.83401887 0.88093856 1.77930996]
1388 ) -22.0 [1.94234243 0.20833352 0.48993456]
1389 ) -12.0 [1.87036589 0.96900669 6.23245296]
1390 ) -2.0 [2.58685232 0.85126352 1.5778268 ]
1391 ) -5 [2.85264118 0.75134696 1.9562271 ]
1392 ) 2.0 [3.16562561

In [76]:
reward

894.0

In [33]:
env.render()

(0.8258704620451992, 0.8180095103291405, 0.11367700334645825)

In [34]:
a

array([-2.22044605e-16])

In [18]:
def random_aim():
    aim = np.random.random(2)
    return aim/aim.sum()

In [26]:
np.exp(1)

2.718281828459045

# GAZEBO

In [26]:
env1 = gym.make("crumb-pick-v0")

In [27]:
sign = lambda a: 1 if a>0 else -1 if (a<0) else 0
metric = lambda a1, a2: ((a1[0] - a2[0])**2 + (a1[1] - a2[1])**2)**(1/2)
def signs(vec_a):
    signs = np.zeros(len(vec_a))
    for i in range(len(vec_a)):
        signs[i] = sign(vec_a[i])
    return signs
        
        
def module(vec_a):
    return (vec_a**2).sum()**(0.5)

def angle(vec_a, vec_b):
    return np.arccos((vec_a*vec_b).sum(axis = 1)/(module(vec_a) * module(vec_b)))
def norm(vec_a):
    norm = np.zeros_like(vec_a)
    for i in range(len(vec_a)):
        norm[i][0] = -vec_a[i][1]
        norm[i][1] = vec_a[i][0]
    return norm
        
def get_xz(joint):
    a = env1.link_state(joint, '').link_state.pose.position
    return np.array([a.x, a.z])

def get_xy(joint):
    a = env1.link_state(joint, '').link_state.pose.position
    return np.array([a.x, a.y])

def state1(aim = env1.aim):
    aim1 = np.array([aim.x, aim.z])
    aim1_gor = np.array([aim.x, aim.y])

    #Joints pose vert
    joint = np.zeros((6,2))
    joint[0] = get_xz('biceps_link')
    joint[1] = get_xz('forearm_link')
    joint[2] = get_xz('wrist_1_link')
    joint[3] = get_xz('gripper_1_link')
    
    #Joints pose gor
    joint[4] = get_xy('gripper_1_link')
    joint[5] = get_xy('biceps_link')
    
    #vectors
    vec1 = np.zeros((3,2))
    vec2 = np.zeros((3,2))
    vec3 = np.zeros((1,2))
    vec4 = np.zeros((1,2))
    
    vec3[0] = joint[4] - joint[5]
    vec4[0] = aim1_gor - joint[5]
    for i in range(3):
        vec1[i] = joint[3] - joint[i]
        vec2[i] = aim1 - joint[i]
        
    
    
    state1 = angle(norm(vec1), vec2) - np.full(3, radians(90))
    state2 = signs(angle(norm(vec3), vec4) - np.full(3, radians(90))) #вестор из 1, -1, 0
    return (tuple(state1),  metric(aim1, joint[3]), tuple(state2))

In [45]:
state = state1(env1.reset())[0]
action, _, _ = act(state)
env1.step([])

KeyboardInterrupt: 

In [28]:
a, _, _ = env1.step([1,radians(-90)])

In [52]:
state = state1()

In [46]:
a

0.009780860148758563

In [53]:
action, _,_ = act((state[1],)+ state[0], sample = False)

In [54]:
a, _, _ = env1.step([1, a+radians(action[0])])

In [51]:
a = env1.reset()[1]

In [56]:
radians(action)

-0.002352551407898241